In [1]:
print("Hello World")

Hello World


The next stage of the project is looking at the algorithims, and working with a reduced set of dimensions, looking at imporving our random forest and trying other algorithims

In [2]:

paths=['Data/parquet/1.Deauth',
       'Data/parquet/2.Disas',
       'Data/parquet/3.(Re)Assoc',
       'Data/parquet/4.Rogue_AP',
       'Data/parquet/5.Krack',
       'Data/parquet/6.Kr00k',
       'Data/parquet/7.SSH',
       'Data/parquet/8.Botnet',
       'Data/parquet/9.Malware',
       'Data/parquet/10.SQL_Injection',
       'Data/parquet/11.SSDP',
       'Data/parquet/12.Evil_Twin',
       'Data/parquet/13.Website_spoofing'
      ]

In [3]:
from pyspark.sql import SparkSession
import seaborn as sns
import matplotlib.pyplot as plt 
from pyspark.sql.functions import col

In [4]:
spark = SparkSession.builder \
    .master("local") \
    .appName("WiFi") \
    .config("spark.driver.memory", "4g")\
    .getOrCreate()

In [5]:
df=spark.read.parquet(*paths)

In [6]:
df.groupBy('StringLabel').count().show()
df.count()

+----------------+--------+
|     StringLabel|   count|
+----------------+--------+
|          Normal|30387099|
|           Krack|   49990|
|          Botnet|   56891|
|   SQL_Injection|    2629|
|       (Re)Assoc|    5502|
|             SSH|   11882|
|       Evil_Twin|  104827|
|Website_spoofing|  405121|
|              NA|      34|
|         Malware|  131611|
|          Deauth|   38942|
|         RogueAP|    1310|
|            SSDP| 5499851|
|           Kr00k|  191803|
|           Disas|   75131|
+----------------+--------+



36962623

https://docs.microsoft.com/en-us/azure/databricks/_static/notebooks/getting-started/get-started-with-mllib-dbr7.html as a guide

We want to use a stratified subsample, we need a small sample for the grid search, and a larger group to test and train on. This is to allow experimentation with results in a reasonable time. Once we have this infomation we can run on the full dataset either in the cloud or locally overnight. I found when working locally on my machine the higher entropy of using multiple attacks led to longer run times, with the tree bagging scores getting large and pushing the data out of memory and causing thrashing. I selected my sample wanting to get a sample of all the attacks but also bearing in mind the power of my computer which is a fairly powerful gaming laptop.  I aimed for about 3,000 of each attack, with a fraction at one significany figure,althou some of the attacks like SQL injection and Rogue AP have fewer attack packets than this. From test runs I found this allowed me to run a random forest locally in about 10-30 minutes which allows me to tune and test the algorithims.

The official page of the random forest algorithim says "Running on a data set with 50,000 cases and 100 variables, it produced 100 trees in 11 minutes on a 800Mhz machine" This is about the run time I am looking for testing. My sample is a lot bigger but I will use fewer variables. 

For this we want the majority of packets to be normal. Some of the attacks (such as Website Spoofing and SSDP have a large percentage and large number of attack packets). I want my algorthims not to favour identifying these attacks, both because I do not have a reason to think they are more prevelant in the real world and because the management packet attacks are more useful to study. I mean this in the sense that the random forest is identifying the attack packet by packet type rather than the attacker by IP or MAC address.





https://spark.apache.org/docs/3.1.1/api/python/reference/api/pyspark.sql.DataFrame.sampleBy.html

In [9]:
subSample = df.sampleBy("StringLabel", fractions={"Normal": 0.02, "Krack": 0.05 ,"Botnet": 0.05 ,"SQL_Injection": 1,"(Re)Assoc": 0.6,"SSH": 0.3,"Evil_Twin": 0.03,"Website_spoofing": 0.006,"Malware": 0.02,"Deauth": 0.06,"RogueAP": 1,"SSDP": 0.0005,"Kr00k": 0.02,"Disas": 0.04, }, seed=0)
subSample.groupBy('StringLabel').count().show()
subSample.groupBy('label').count().show()
subSample.count()

+----------------+------+
|     StringLabel| count|
+----------------+------+
|          Normal|608992|
|           Krack|  2514|
|          Botnet|  2852|
|   SQL_Injection|  2629|
|       (Re)Assoc|  3413|
|             SSH|  3534|
|       Evil_Twin|  3105|
|Website_spoofing|  2544|
|         Malware|  2679|
|          Deauth|  2220|
|         RogueAP|  1310|
|           Kr00k|  3790|
|           Disas|  2977|
|            SSDP|  2755|
+----------------+------+

+-----+------+
|label| count|
+-----+------+
|    0|608992|
|    1| 36322|
+-----+------+



645314

So here we are left with 645314 events and 94% Normal packets. 

We then want to write out our dataframe as a parquet file. This is so when we load the data we avoid data from the original dataframe staying in memory making basic calculations more time consuming. Spark can work with this whole dataset and much larger ones too, but the run time isn't necessarily appropriate for me as a part time masters student.

In [10]:
writePath="Data/parquet/subSampleWholeDataSet"


In [12]:
subSample.write.parquet(writePath)